In [ ]:
# Problems Notebook

In [ ]:
## Problem 1: Binary Words and Operations

### Introduction

SHA-256 (Secure Hash Algorithm 256-bit) is a cryptographic hash function that is part of the SHA-2 family, standardized by NIST in the [Federal Information Processing Standards Publication 180-4](link). The SHA-256 algorithm operates on 32-bit words (binary strings of length 32) and uses several bitwise logical functions and rotation operations to process input data.

In [ ]:
In this problem, we implement seven key functions defined on page 10 of the Secure Hash Standard:

**Logical Functions:**
- **Parity(x, y, z)** - Returns the bitwise XOR of three inputs
- **Ch(x, y, z)** - "Choose" function: uses x to choose bits from y or z
- **Maj(x, y, z)** - "Majority" function: returns the majority bit value at each position

**Rotation Functions:**
- **Σ₀²⁵⁶(x)** - Sigma0: combines three right rotations of x
- **Σ₁²⁵⁶(x)** - Sigma1: combines three right rotations of x  
- **σ₀²⁵⁶(x)** - sigma0: combines rotations and shifts for message schedule
- **σ₁²⁵⁶(x)** - sigma1: combines rotations and shifts for message schedule

All operations are performed on 32-bit unsigned integers, and we use NumPy's `uint32` data type to ensure proper handling of overflow and bitwise operations.

In [ ]:
### Implementation

In [ ]:
#### 1. Parity Function

The Parity function computes the bitwise XOR (exclusive OR) of three 32-bit words. According to the Secure Hash Standard (page 10), the Parity function is defined as:

**Parity(x, y, z) = x ⊕ y ⊕ z**

Where ⊕ represents the bitwise XOR operation. This function returns 1 for each bit position where an odd number of the inputs have a 1 bit, and 0 otherwise. The Parity function is used in SHA-1 for certain rounds of the compression function.

In [5]:
import numpy as np

def Parity(x, y, z):
    """
    Compute the bitwise Parity of three 32-bit words.
    
    The Parity function returns the bitwise XOR (exclusive OR) of x, y, and z.
    This is a logical function used in SHA-1 hash algorithm.
    
    Parameters
    ----------
    x : int or numpy.uint32
        First 32-bit word
    y : int or numpy.uint32
        Second 32-bit word
    z : int or numpy.uint32
        Third 32-bit word
    
    Returns
    -------
    numpy.uint32
        The bitwise XOR of x, y, and z
        
    Examples
    --------
    >>> Parity(0b1100, 0b1010, 0b1001)
    7
    >>> Parity(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF)
    4294967295
    """
    # Ensure all inputs are treated as 32-bit unsigned integers
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    # Compute bitwise XOR: x ⊕ y ⊕ z
    return x ^ y ^ z

In [ ]:
The implementation uses NumPy's `uint32` type to ensure all values are treated as 32-bit unsigned integers. The XOR operator `^` in Python performs bitwise XOR on each bit position independently.

In [ ]:
##### Testing Parity Function

In [7]:
# Test 1: binary example
# 1100 XOR 1010 = 0110, then 0110 XOR 1001 = 1111 (binary) = 15 (decimal)
result1 = Parity(0b1100, 0b1010, 0b1001)
print(f"Test 1: Parity(0b1100, 0b1010, 0b1001) = {result1}")
print(f"Expected: 15, Got: {result1}, Pass: {result1 == 15}")

Test 1: Parity(0b1100, 0b1010, 0b1001) = 15
Expected: 15, Got: 15, Pass: True


In [8]:
# Test 2: All bits set
# When all three inputs have all bits set, XOR returns all bits set
result2 = Parity(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF)
print(f"\nTest 2: Parity(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF) = {result2}")
print(f"Expected: {np.uint32(0xFFFFFFFF)}, Got: {result2}, Pass: {result2 == np.uint32(0xFFFFFFFF)}")


Test 2: Parity(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF) = 4294967295
Expected: 4294967295, Got: 4294967295, Pass: True


In [9]:
# Test 3: Zero inputs
# XOR of zeros is zero
result3 = Parity(0, 0, 0)
print(f"\nTest 3: Parity(0, 0, 0) = {result3}")
print(f"Expected: 0, Got: {result3}, Pass: {result3 == 0}")


Test 3: Parity(0, 0, 0) = 0
Expected: 0, Got: 0, Pass: True


In [10]:
# Test 4: Identity property - XOR with two identical values
# x XOR x XOR y = y (since x XOR x = 0)
x_val = np.uint32(0xABCDEF01)
y_val = np.uint32(0x12345678)
result4 = Parity(x_val, x_val, y_val)
print(f"\nTest 4: Parity(0xABCDEF01, 0xABCDEF01, 0x12345678) = {hex(result4)}")
print(f"Expected: {hex(y_val)}, Got: {hex(result4)}, Pass: {result4 == y_val}")


Test 4: Parity(0xABCDEF01, 0xABCDEF01, 0x12345678) = 0x12345678
Expected: 0x12345678, Got: 0x12345678, Pass: True


In [ ]:
#### 2. Ch (Choice) Function

The Ch function is a conditional function that "chooses" bits from y or z based on the corresponding bit in x. According to the Secure Hash Standard the Ch function is defined as:

**Ch(x, y, z) = (x ∧ y) ⊕ (¬x ∧ z)**

Where ∧ represents bitwise AND, ⊕ represents bitwise XOR, and ¬ represents bitwise NOT (complement).

The function works as follows: for each bit position, if the bit in x is 1, the result takes the bit from y; if the bit in x is 0, the result takes the bit from z. This is why it's called the "choice" function - x chooses between y and z.

In [15]:
def Ch(x, y, z):
    """
    Compute the Ch (Choice) function of three 32-bit words.
    
    The Ch function uses x as a selector to choose bits from either y or z.
    For each bit position: if x bit is 1, choose y bit; if x bit is 0, choose z bit.
    
    Formula: Ch(x, y, z) = (x ∧ y) ⊕ (¬x ∧ z)
    
    Parameters
    ----------
    x : int or numpy.uint32
        Selector word (32-bit)
    y : int or numpy.uint32
        First choice word (32-bit)
    z : int or numpy.uint32
        Second choice word (32-bit)
    
    Returns
    -------
    numpy.uint32
        Result of the choice function
        
    Examples
    --------
    >>> Ch(0b1111, 0b1010, 0b0101)
    10
    >>> Ch(0xFFFFFFFF, 0x12345678, 0xABCDEF01)
    305419896
    """
    # Ensure all inputs are treated as 32-bit unsigned integers
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    # Compute Ch(x, y, z) = (x ∧ y) ⊕ (¬x ∧ z)
    return (x & y) ^ (~x & z)

In [ ]:
The implementation follows the standard formula directly:
- `(x & y)`: selects bits from y where x has 1s
- `(~x & z)`: selects bits from z where x has 0s
- The XOR combines these two results

Since the two AND operations produce non-overlapping bit patterns (where x is 1 vs where x is 0), the XOR effectively acts as an OR, merging the selected bits.

In [ ]:
##### Testing Ch Function


In [16]:
# Test 1: Simple example where x chooses between y and z
# x = 1111 (all 1s) should select all bits from y = 1010
# Result should be 1010 (binary) = 10 (decimal)
result1 = Ch(0b1111, 0b1010, 0b0101)
print(f"Test 1: Ch(0b1111, 0b1010, 0b0101) = {result1}")
print(f"Expected: 10 (0b1010), Got: {result1}, Pass: {result1 == 10}")

Test 1: Ch(0b1111, 0b1010, 0b0101) = 10
Expected: 10 (0b1010), Got: 10, Pass: True


In [17]:
# Test 2: x = 0000 (all 0s) should select all bits from z = 0101
# Result should be 0101 (binary) = 5 (decimal)
result2 = Ch(0b0000, 0b1010, 0b0101)
print(f"\nTest 2: Ch(0b0000, 0b1010, 0b0101) = {result2}")
print(f"Expected: 5 (0b0101), Got: {result2}, Pass: {result2 == 5}")


Test 2: Ch(0b0000, 0b1010, 0b0101) = 5
Expected: 5 (0b0101), Got: 5, Pass: True


In [18]:
# Test 3: Mixed selection
# x = 1100, y = 1010, z = 0101
# Bits 0-1: x=00, select from z=01 -> 01
# Bits 2-3: x=11, select from y=10 -> 10
# Result: 1001 (binary) = 9 (decimal)
result3 = Ch(0b1100, 0b1010, 0b0101)
print(f"\nTest 3: Ch(0b1100, 0b1010, 0b0101) = {result3}")
print(f"Expected: 9 (0b1001), Got: {result3}, Pass: {result3 == 9}")


Test 3: Ch(0b1100, 0b1010, 0b0101) = 9
Expected: 9 (0b1001), Got: 9, Pass: True


In [19]:
# Test 4: Full 32-bit values
# When x = all 1s, result equals y
x_val = np.uint32(0xFFFFFFFF)
y_val = np.uint32(0x12345678)
z_val = np.uint32(0xABCDEF01)
result4 = Ch(x_val, y_val, z_val)
print(f"\nTest 4: Ch(0xFFFFFFFF, 0x12345678, 0xABCDEF01) = {hex(result4)}")
print(f"Expected: {hex(y_val)}, Got: {hex(result4)}, Pass: {result4 == y_val}")


Test 4: Ch(0xFFFFFFFF, 0x12345678, 0xABCDEF01) = 0x12345678
Expected: 0x12345678, Got: 0x12345678, Pass: True


In [20]:
# Test 5: When x = all 0s, result equals z
x_val = np.uint32(0x00000000)
y_val = np.uint32(0x12345678)
z_val = np.uint32(0xABCDEF01)
result5 = Ch(x_val, y_val, z_val)
print(f"\nTest 5: Ch(0x00000000, 0x12345678, 0xABCDEF01) = {hex(result5)}")
print(f"Expected: {hex(z_val)}, Got: {hex(result5)}, Pass: {result5 == z_val}")


Test 5: Ch(0x00000000, 0x12345678, 0xABCDEF01) = 0xabcdef01
Expected: 0xabcdef01, Got: 0xabcdef01, Pass: True


In [ ]:
#### 3. Maj (Majority) Function

The Maj function returns the majority value for each bit position across three 32-bit words. According to the Secure Hash Standard (page 10), the Maj function is defined as:

**Maj(x, y, z) = (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)**

Where ∧ represents bitwise AND and ⊕ represents bitwise XOR.

For each bit position, the function returns 1 if at least two of the three input bits are 1, and returns 0 otherwise. This implements a majority vote at each bit position independently.

In [ ]:
def Maj(x, y, z):
    """
    Compute the Maj (Majority) function of three 32-bit words.
    
    The Maj function returns the majority bit value at each bit position.
    If at least two of the three bits are 1, the result is 1; otherwise 0.
    
    Formula: Maj(x, y, z) = (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)
    
    Parameters
    ----------
    x : int or numpy.uint32
        First 32-bit word
    y : int or numpy.uint32
        Second 32-bit word
    z : int or numpy.uint32
        Third 32-bit word
    
    Returns
    -------
    numpy.uint32
        Result where each bit is the majority of the corresponding input bits
        
    Examples
    --------
    >>> Maj(0b1110, 0b1100, 0b1000)
    12
    >>> Maj(0xFFFFFFFF, 0xFFFFFFFF, 0x00000000)
    4294967295
    """
    # Ensure all inputs are treated as 32-bit unsigned integers
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    # Compute Maj(x, y, z) = (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)
    return (x & y) ^ (x & z) ^ (y & z)